<a href="https://colab.research.google.com/github/cherlim/HWR/blob/main/handwritten_digit_gradio_CherV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Handwriting digit recognition
**Name: Lim Cher Eng**

In [1]:
!pip install -q gradio

     |████████████████████████████████| 3.4 MB 11.6 MB/s 
     |████████████████████████████████| 1.9 MB 39.0 MB/s 
     |████████████████████████████████| 206 kB 66.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 961 kB 53.6 MB/s 
     |████████████████████████████████| 3.5 MB 60.5 MB/s 


In [2]:
import cv2
import gradio as gr

In [3]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0, 
x_test = x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2740 - accuracy: 0.9218
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1240 - accuracy: 0.9635
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0879 - accuracy: 0.9736
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0673 - accuracy: 0.9790
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0520 - accuracy: 0.9844
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0423 - accuracy: 0.9868
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0340 - accuracy: 0.9892
Epoch 8/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0280 - accuracy: 0.9913
Epoch 9/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0232 - accuracy: 0.9931
Epoch 10/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.019

In [5]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 0s 893us/step - loss: 0.0895 - accuracy: 0.9758


[0.08951474726200104, 0.9757999777793884]

In [6]:
model_name = 'digits_recognition_dnn.h5'
#model.save(model_name, save_format='h5')
model_dnn = tf.keras.models.load_model(model_name)

In [7]:
model_name = 'digits_recognition_cnn.h5'
model_cnn = tf.keras.models.load_model(model_name)

# Center input image, fill up broken line, round off edges

In [8]:
import numpy as np
import urllib.request

#urllib.request.urlretrieve('https://github.com/cherlim/NP/blob/main/digits_recognition_cnn.h5', 'cnn.h5')
#MODEL_PATH = './cnn.h5'
#urllib.request.urlretrieve('https://github.com/cherlim/NP/blob/main/digits_recognition_dnn.h5', 'dnn.h5')
#MODEL_PATH = './dnn.h5'

# Using moments to center image

In [9]:
def center_image(image):
    height, width = image.shape
  
    print(image.shape)
    wi=(width/2)
    he=(height/2)
    print(wi,he)

# get contours (presumably just one around the nonzero pixels) 
    kernel = np.ones((5,5),np.uint8)
# Dilate: morph input image with kernel to fill in the gaps
    img_dilate=cv2.dilate(image,kernel,iterations = 1)
# Erode the edges to reduce thickness
    img_erode=cv2.erode(img_dilate,kernel,iterations = 1)
# Set pixel intensity less than input threshold (=95) to 0 
# CV2 threshold fn returns 2 outputs: ret=threshold; thresh = thresholded image
    ret,thresh = cv2.threshold(img_erode,95,255,0) 

    M = cv2.moments(thresh)

    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    offsetX = (wi-cX)
    offsetY = (he-cY)
    T = np.float32([[1, 0, offsetX], [0, 1, offsetY]]) 
    centered_image = cv2.warpAffine(img_erode, T, (width, height))

    return centered_image

In [10]:
def modelC(image):
    image_c = center_image(image)
    image_new = image_c.reshape(1, 28, 28, 1) # CNN model input layer shape
    #print("after reshape 1,28,28,1 ", image_new.shape)
    prediction_cnn = model_cnn.predict(image_new).tolist()[0]
    return {str(i): prediction_cnn[i] for i in range(10)}

In [11]:
def modelD(image):
    image_c = center_image(image)
    image_new = image_c.reshape(1, 28, 28, 1) # DNN model input layer shape
    #print("after reshape 1,28,28,1 ", image_new.shape)
    prediction_dnn = model_dnn.predict(image_new).tolist()[0]
    return {str(i): prediction_dnn[i] for i in range(10)}

In [12]:
label = gr.outputs.Label(num_top_classes = 3) # initializing the output component
# launching the interface
gr.Interface([modelC, modelD], inputs = "sketchpad",outputs = label).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://58826.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://58826.gradio.app')

# Using bounding box to center image

In [13]:
def center_imageB(image):
    height, width = image.shape
  
    print(image.shape)
    wi=(width/2)
    he=(height/2)
    print(wi,he)

# get contours (presumably just one around the nonzero pixels) 
    kernel = np.ones((5,5),np.uint8)
# Dilate: morph input image with kernel to fill in the gaps
    img_dilate=cv2.dilate(image,kernel,iterations = 1)
# Erode the edges to reduce thickness
    img_erode=cv2.erode(img_dilate,kernel,iterations = 1)

    contours = cv2.findContours(img_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
    
    # recenter image
    startx = (width - w)//2
    starty = (height - h)//2
    result = np.zeros_like(image)
    result[starty:starty+h,startx:startx+w] = image[y:y+h,x:x+w]

    return result

In [14]:
def modelBC(image):
    image_c = center_imageB(image)
    image_new = image_c.reshape(1, 28, 28, 1) # CNN model input layer shape
    #print("after reshape 1,28,28,1 ", image_new.shape)
    prediction_cnn = model_cnn.predict(image_new).tolist()[0]
    return {str(i): prediction_cnn[i] for i in range(10)}

In [15]:
def modelBD(image):
    image_c = center_imageB(image)
    image_new = image_c.reshape(1, 28, 28, 1) # DNN model input layer shape
    #print("after reshape 1,28,28,1 ", image_new.shape)
    prediction_dnn = model_dnn.predict(image_new).tolist()[0]
    return {str(i): prediction_dnn[i] for i in range(10)}

In [16]:
label = gr.outputs.Label(num_top_classes = 3) # initializing the output component
# launching the interface
gr.Interface([modelBC,modelBD], inputs = "sketchpad",outputs = label).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://33681.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://33681.gradio.app')

# Combining both contour and moments to center the image

**"A Simple and Effective Optical Character Recognition System for Digits Recognition using the Pixel-Contour Features and Mathematical Parameters"** 

Ref: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.660.1825&rep=rep1&type=pdf 

In [17]:
def center_imageBM(image):
    height, width = image.shape
  
    print(image.shape)
    wi=(width/2)
    he=(height/2)
    print(wi,he)

# get contours (presumably just one around the nonzero pixels) 
    kernel = np.ones((5,5),np.uint8)
# Dilate: morph input image with kernel to fill in the gaps
    img_dilate=cv2.dilate(image,kernel,iterations = 1)
# Erode the edges to reduce thickness
    img_erode=cv2.erode(img_dilate,kernel,iterations = 1)

    contours = cv2.findContours(img_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
    
    # recenter image
    startx = (width - w)//2
    starty = (height - h)//2
    result = np.zeros_like(image)
    result[starty:starty+h,startx:startx+w] = image[y:y+h,x:x+w]

    # CV2 threshold fn returns 2 outputs: ret=threshold; thresh = thresholded image
    #ret,thresh = cv2.threshold(img_erode,95,255,0) 

    M = cv2.moments(result)

    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    offsetX = (wi-cX)
    offsetY = (he-cY)
    T = np.float32([[1, 0, offsetX], [0, 1, offsetY]]) 
    centered_image = cv2.warpAffine(result, T, (width, height))



    return centered_image

In [18]:
def modelBMC(image):
    image_c = center_imageBM(image)
    image_new = image_c.reshape(1, 28, 28, 1) # CNN model input layer shape
    #print("after reshape 1,28,28,1 ", image_new.shape)
    prediction_cnn = model_cnn.predict(image_new).tolist()[0]
    return {str(i): prediction_cnn[i] for i in range(10)}

In [19]:
def modelBMD(image):
    image_c = center_imageBM(image)
    image_new = image_c.reshape(1, 28, 28, 1) # DNN model input layer shape
    #print("after reshape 1,28,28,1 ", image_new.shape)
    prediction_dnn = model_dnn.predict(image_new).tolist()[0]
    return {str(i): prediction_dnn[i] for i in range(10)}

In [20]:
label = gr.outputs.Label(num_top_classes = 3) # initializing the output component
# launching the interface
gr.Interface([modelBMC,modelBMD], inputs = "sketchpad",outputs = label).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://23790.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7862/',
 'https://23790.gradio.app')